## Elo Rating method

This is the Elo method.  The example is taken from "Who's #1" by Langville and Meyer and the results match.  Note, the data includes the entire 2009 NFL season, including all playoff games.  

Reference: https://www.gautamnarula.com/rating/

In [7]:
import math 
  
# Function to calculate the Probability 
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 1000)) 
  
# Function to calculate Elo rating 
# K is a constant. 
# Player A wins over Player B.  
# tie = true if tie, false otherwise
def EloRating(Ra, Rb, K, tie): 
    
    # To calculate the Winning 
    # Probability of Player B 
    Pb = Probability(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = Probability(Rb, Ra) 
  
    # Updating the Elo Ratings 
    if tie:
       Ra = Ra + K * (1/2 - Pa) 
       Rb = Rb + K * (1/2 - Pb) 
    else:        
       Ra = Ra + K * (1 - Pa) 
       Rb = Rb + K * (0 - Pb) 
    
    return Ra, Rb

In [8]:
gameFilename = '2015APseedgame.csv'
teamFilename = '2015player.csv'

### Load the team names into an array

In [9]:
import pandas as pd

teamNames = pd.read_csv(teamFilename, header = None)
numTeams = len(teamNames)

### Load the games

In [10]:
# columns of games are:
#	column 0 = days since 1/1/0000
#	column 1 = date in YYYYMMDD format
#	column 2 = team1 index
#	column 3 = team1 homefield (1 = home, -1 = away, 0 = neutral)
#	column 4 = team1 score
#	column 5 = team2 index
#	column 6 = team2 homefield (1 = home, -1 = away, 0 = neutral)
#	column 7 = team2 score
games = pd.read_csv(gameFilename, header = None)
games[0].replace(365243, 737600, inplace=True)
numGames = len(games)

### Create Elo ratings

In [11]:
import numpy as np

print(">>>>>>> I'm not including ties.")

eloRatings = np.zeros(numTeams)

for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 # subtracting 1 since python indexes at 0
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 # subtracting 1 since python indexes at 0
    team2Score = games.loc[i, 7]

    if team1Score > team2Score:
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID],32, False)
    elif team1Score < team2Score:
        team2Rating, team1Rating = EloRating(eloRatings[team2ID], eloRatings[team1ID],32, False)
    else:  
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID],32, True)
        
    eloRatings[team1ID] = team1Rating
    eloRatings[team2ID] = team2Rating

>>>>>>> I'm not including ties.


### Sort and print the ranking of teams

In [12]:
iSort = np.argsort(-eloRatings)

print('\n\n************** ELO Rating Method **************\n')
print('===========================')
print('Rank   Rating      Team   ')
print('===========================')
for i in range(numTeams):
    print(f'{i+1:4d}   {eloRatings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')

print('')   # extra carriage return



************** ELO Rating Method **************

Rank   Rating      Team   
   1   645.47399  McIlroy
   2   591.42247  Spieth
   3   556.63175  Watson
   4   542.67733  Stenson
   5   540.52318  Furyk
   6   524.10718  Rose
   7   487.34428  Johnson
   8   442.38641  Day
   9   423.17022  Fowler
  10   416.84895  García
  11   416.41866  Holmes
  12   405.20547  Scott
  13   374.40022  Matsuyama
  14   367.50852  Walker
  15   355.73492  Reed
  16   343.30143  Kuchar
  17   314.24031  Horschel
  18   295.64266  Kaymer
  19   289.38049  Koepka
  20   238.89426  Na
  21   216.35036  Haas
  22   199.81277  Johnson
  23   191.30970  Dubuisson
  24   191.04655  Westwood
  25   177.40044  Kirk
  26   174.89070  Palmer
  27   134.45392  Oosthuizen
  28   107.05491  Poulter
  29   89.55204  Mahan
  30   81.26520  Donaldson
  31   51.79643  Moore
  32   -4.63538  Casey
  33   -21.38921  Schwartzel
  34   -23.16791  Lahiri
  35   -24.97643  McDowell
  36   -38.72540  Bradley
  37   -70.39966 

In [13]:
colleyRank = pd.DataFrame(eloRatings)
player = pd.DataFrame(teamNames)
rrank = colleyRank[0]
ewf = player[1]
df = pd.concat([rrank,ewf],axis=1)
dv = df.sort_values(0,ascending=False)
dc = dv.reset_index()
del dc['index']
dc = dc.reset_index()
dc['index'] = dc['index'] + 1
dc = dc. sort_values(1)
dc = dc.rename(columns={'index':'rank'})
dc = dc.rename(columns={1:'teams'})
del dc[0]
dc = dc[['teams','rank']]


In [8]:
de = teamNames.rename(columns={0:'ID', 1:'Name'})
de['ID'].astype(np.int64)
de.set_index('ID')
mydict = dict(zip(de.Name, de.ID))
dc['teams'].replace(mydict, inplace=True)
dc.head()

,teams,rank
23,1,24
19,2,20
0,3,1
9,4,10
26,5,27


In [9]:
def returnRank(t):
    df = dc.loc[dc['teams'] == t]
    return df.iat[0,1]

In [10]:
gamet = pd.read_csv('gamedd.csv', header = None)
weekg = gamet.loc[gamet[0] == 17]
numgweek = len(weekg)
correct = []
incorrect = []
num = 1

In [11]:
for i in range(numgweek):
    tea1ID = weekg.iloc[i, 3]
    tea2ID = weekg.iloc[i, 6]
    tea1Score = weekg.iloc[i, 5]
    tea2Score = weekg.iloc[i, 8]
    tea1Rank = returnRank(tea1ID)
    tea2Rank = returnRank(tea2ID)
    if tea1Score > tea2Score and tea1Rank < tea2Rank:
        correct.append(num)
    else:
        incorrect.append(num)

In [12]:
len(correct)/numgweek

0.5

## 